In [1]:
from cellpose import models, io
from cellpose.io import *
from collections import defaultdict
import geopandas
import glob
import imagej
from jpype import JArray, JInt
import matplotlib.pyplot as plt
import multiprocessing as mp
import numpy as np
import os
import re
import pandas
from pandas import DataFrame
from pathlib import Path
import scyjava
import seaborn
import shutil
import tkinter as tk
from tkinter import filedialog
from PIL import Image

In [2]:
scyjava.config.add_option('-Xmx30g')
start_dir = os.getcwd()
#ij = imagej.init('sc.fiji:fiji', mode='interactive')
ij = imagej.init('/home/saka/sw/local/fiji/2023', mode='interactive')
## Something about this init() function changes the current working directory.
ij.getApp().getInfo(True)
ij.ui().showUI()
os.chdir(start_dir)
ij.getVersion() #This is to make sure ImageJ/Fiji opened properly. In case of error, it could display '2.9.0/inactive' instead of the full version number

'2.9.0/1.54e'

ZeissCZIReader initializing /home/saka/Documents/Lab stuff/confocal/exp2/Experiment-1647-Split Scenes-01.czi
X
Y
Z
T


In [3]:
showPolygonRoi = scyjava.jimport('ij.gui.PolygonRoi')
Overlay = scyjava.jimport('ij.gui.Overlay')
Regions = scyjava.jimport('net.imglib2.roi.Regions')
LabelRegions = scyjava.jimport('net.imglib2.roi.labeling.LabelRegions')
ZProjector = scyjava.jimport('ij.plugin.ZProjector')()
ov = Overlay()

In [4]:
Overlay = scyjava.jimport('ij.gui.Overlay')
ov = Overlay()
rm = ij.RoiManager.getRoiManager()

In [5]:
directory_path = '/home/saka/Documents/Lab stuff/confocal/exp2'
os.chdir(directory_path)

In [6]:
root = tk.Tk()
root.withdraw()
filepath = filedialog.askopenfilename()

In [7]:
image = ij.io().open(filepath)
#ij.ui().show(image)
imp = ij.py.to_imageplus(image)

[java.lang.Enum.toString] ZeissCZIReader initializing /home/saka/Documents/Lab stuff/confocal/exp2/Experiment-1647-Split Scenes-01.czi
[WARN] Unknown DetectorType value 'GaAsP-PMT' will be stored as "Other"
[java.lang.Enum.toString] [WARN] Unknown DetectorType value 'GaAsP-PMT' will be stored as "Other"
[WARN] Unknown DetectorType value 'GaAsP-PMT' will be stored as "Other"
[java.lang.Enum.toString] [WARN] Unknown DetectorType value 'GaAsP-PMT' will be stored as "Other"
[WARN] Unknown DetectorType value 'GaAsP-PMT' will be stored as "Other"
[java.lang.Enum.toString] [WARN] Unknown DetectorType value 'GaAsP-PMT' will be stored as "Other"
[WARN] Unknown DetectorType value 'Multialkali-PMT' will be stored as "Other"
[java.lang.Enum.toString] [WARN] Unknown DetectorType value 'Multialkali-PMT' will be stored as "Other"


In [8]:
wanted_channel = 3
channel = image[:, :, wanted_channel, :, :]
imp = ij.py.to_imageplus(channel)
imp.setDimensions(1, 11, 121)
#ij.ui().show(imp)

In [10]:
data_info = {}
for element in range(len(imp.dims)):
    name = imp.dims[element]
    data_info[name] = imp.shape[element]
    print(name)
num_frame = data_info['T'] + 1

In [11]:
for timepoint in range(1, num_frame):
    frame_number = timepoint - 1
    input_txt = Path(f"{directory_path}/outputs/Experiment-1647-Split Scenes-01_z2/cellpose/frame_{frame_number}_cp_outlines.txt")
    txt_fh = open(input_txt, 'r')
    for line in txt_fh:
            xy = line.rstrip().split(",")
            xy_coords = [int(element) for element in xy if element not in '']
            x_coords = [int(element) for element in xy[::2] if element not in '']
            y_coords = [int(element) for element in xy[1::2] if element not in '']
            xcoords_jint = JArray(JInt)(x_coords)
            ycoords_jint = JArray(JInt)(y_coords)
            polygon_roi_instance = scyjava.jimport('ij.gui.PolygonRoi')
            roi_instance = scyjava.jimport('ij.gui.Roi')
            imported_polygon = polygon_roi_instance(xcoords_jint, ycoords_jint, len(x_coords), int(roi_instance.POLYGON))
            imp.setRoi(imported_polygon)
            rm.addRoi(imported_polygon)
            roi_count = rm.getCount()
            rm.select(roi_count - 1)
            time_set = imp.setT(timepoint)   
            #rm.runCommand('Update')
#ij.py.run_macro("roiManager('Select All');")
f_name = os.path.basename(filepath)
f_name = os.path.splitext(f_name)[0]
rm.runCommand("Save", f"{directory_path}/" + f"{f_name}.zip")

True

In [16]:
ij.ui().show(image)